In [16]:
import pandas as pd
import os
import mygene
from tqdm.notebook import tqdm

In [17]:

mg = mygene.MyGeneInfo()

# Asthma

In [18]:
asthma_targets = pd.read_csv("data/GWAS_catalog/asthma/asthma_loci_gwascatalog_1933_assoc_24Jan22.csv")
asthma_targets

,Variant and risk allele,P-value,P-value.1,P-value annotation,RAF,OR,Beta,CI,Mapped gene,Reported trait,Trait(s),Background trait(s),Study accession,Location
0,rs9866261-<b>G</b>,1 x 10-7,1.000000e-07,NaN,0.125,1.45,'-,[1.26-1.66],ADAMTS9,Asthma,asthma,'-,GCST003176,3:64540891
1,rs10197862-<b>?</b>,2 x 10-6,2.000000e-06,NaN,NR,1.99,'-,[NR],"IL18R1, IL1RL1",Asthma,asthma,'-,GCST003176,2:102350089
2,rs4795397-<b>A</b>,4 x 10-12,4.000000e-12,NaN,0.64,'-,'-,'-,"ZPBP2, IKZF3",Asthma,asthma,'-,GCST004390,17:39867492
3,rs11078927-<b>C</b>,1 x 10-12,1.000000e-12,(Children),0.57,1.52,'-,'-,GSDMB,Asthma,asthma,'-,GCST004390,17:39908152
4,rs907092-<b>G</b>,1 x 10-12,1.000000e-12,NaN,0.66,1.41,'-,'-,IKZF3,Asthma,asthma,'-,GCST004390,17:39766006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1928,rs697852-<b>?</b>,7 x 10-8,7.000000e-08,NaN,NR,1.0416667,'-,NR,ITPKB,"Allergic disease (asthma, hay fever or eczema)","Eczema, allergic rhinitis, asthma",'-,GCST009716,1:226727033
1929,rs3787184-<b>?</b>,8 x 10-8,8.000000e-08,NaN,NR,1.0386373,'-,NR,NFATC2,"Allergic disease (asthma, hay fever or eczema)","Eczema, allergic rhinitis, asthma",'-,GCST009716,20:51541298
1930,rs2013724-<b>?</b>,1 x 10-8,1.000000e-08,NaN,NR,'-,'-,NR,"NFKBIA, PSMA6","Allergic disease (asthma, hay fever or eczema)","Eczema, allergic rhinitis, asthma",'-,GCST009716,14:35387142
1931,rs1057258-<b>?</b>,3 x 10-7,3.000000e-07,NaN,NR,1.0309278,'-,NR,INPP5D,"Allergic disease (asthma, hay fever or eczema)","Eczema, allergic rhinitis, asthma",'-,GCST009716,2:233206983


In [19]:
asthma_genes = set()
for i, row in asthma_targets.iterrows():
    gns = row["Mapped gene"].split(", ")
    for gn in gns:
        asthma_genes.add(gn)
asthma_genes = sorted(list(asthma_genes))

In [20]:
gene_mapping = mg.querymany(asthma_genes, scopes=["symbol", "alias"], species="human", returnall=True)
gene_mapping["out"][:3]

querying 1-899...done.
Finished.
209 input query terms found dup hits:
	[('ABO', 3), ('ADAMTSL4-AS2', 2), ('ADGRA1-AS1', 2), ('AK4P4', 2), ('ALDH7A1P4', 2), ('ARMC10P1', 2)
1 input query terms found no hit:
	["'-"]


[{'query': "'-", 'notfound': True},
 {'query': 'ABO',
  '_id': 'ENSG00000175164',
  '_score': 96.85642,
  'name': 'ABO, alpha 1-3-N-acetylgalactosaminyltransferase and alpha 1-3-galactosyltransferase',
  'symbol': 'ABO',
  'taxid': 9606},
 {'query': 'ABO',
  '_id': 'ENSG00000281879',
  '_score': 96.85642,
  'name': 'ABO, alpha 1-3-N-acetylgalactosaminyltransferase and alpha 1-3-galactosyltransferase',
  'symbol': 'ABO',
  'taxid': 9606}]

In [21]:
ncbi_ids = []
gene2ncbi = {}
for gn in tqdm(asthma_genes):
    res = [e for e in gene_mapping["out"] if e["query"]==gn]
    res = [r for r in res if "_score" in r.keys() and "entrezgene" in r.keys()]
    if len(res)<1:
#         ncbi_ids.append(None)
        continue
#     elif len(res)>1:
#         break
        
    res = sorted(res, key=lambda x: x["_score"], reverse=True)
    ncbi_ids.append(res[0]["entrezgene"])
    gene2ncbi[gn] = res[0]["entrezgene"]
len(ncbi_ids)

  0%|          | 0/899 [00:00<?, ?it/s]

896

In [22]:
with open("processed_data/gwas_catalog_targets/asthma_targets_ncbi_gwas_catalog.json", "w") as f:
    json.dump(ncbi_ids, f)
with open("processed_data/gwas_catalog_targets/asthma_targets_gene2ncbi.json", "w") as f:
    json.dump(gene2ncbi, f, indent=2)
# ncbi_ids

# Autism

In [23]:
targets = pd.read_csv("data/GWAS_catalog/autism/efotraits_EFO_0003756-associations-2022-03-25.csv")
targets

,Variant and risk allele,P-value,P-value annotation,RAF,OR,Beta,CI,Mapped gene,Reported trait,Trait(s),Background trait(s),Study accession,Location
0,rs9573902-<b>C</b>,9 x 10-6,NaN,'-,'-,'-,'-,"KCTD12, RN7SL571P",Autism,autism,'-,GCST012498,13:76644438
1,chr20:44957939-<b>T</b>,9 x 10-6,NaN,'-,'-,'-,'-,'-,Autism,autism,'-,GCST012498,Mapping not available
2,"rs2280840-<b>G</b>, rs6599528-<b>A</b>",9 x 10-6,NaN,0.625,'-,'-,'-,SLC52A2; FBXL6; No mapped genes,Autism,autism,'-,GCST010733,8:144357897
3,"rs1545620-<b>G</b>, rs7248508-<b>C</b>",9 x 10-6,NaN,0.426,'-,'-,'-,MYO9B; No mapped genes,Autism,autism,'-,GCST010733,19:17192965
4,rs79940520-<b>G</b>,9 x 10-6,NaN,0.141,1.0893246,'-,[NR],"RN7SKP222, FGF12",Autism spectrum disorder,autism spectrum disorder,'-,GCST007556,3:192120380
...,...,...,...,...,...,...,...,...,...,...,...,...,...
796,rs75782365-<b>T</b>,8 x 10-24,NaN,'-,1.21,'-,[1.16-1.25],BTN3A1,Autism spectrum disorder or schizophrenia,"autism spectrum disorder, schizophrenia",'-,GCST004521,6:26408323
797,rs141342723-<b>T</b>,4 x 10-24,NaN,'-,1.2,'-,[1.16-1.24],"RPL8P1, LINC01012",Autism spectrum disorder or schizophrenia,"autism spectrum disorder, schizophrenia",'-,GCST004521,6:27654305
798,rs116137698-<b>A</b>,3 x 10-26,NaN,'-,1.22,'-,[1.17-1.26],"RPSAP2, LINC00533",Autism spectrum disorder or schizophrenia,"autism spectrum disorder, schizophrenia",'-,GCST004521,6:28699892
799,rs115329265-<b>A</b>,2 x 10-27,NaN,'-,1.17,'-,[1.14-1.21],"NOP56P1, RPSAP2",Autism spectrum disorder or schizophrenia,"autism spectrum disorder, schizophrenia",'-,GCST004521,6:28744470


In [24]:
autism_genes = set()
for i, row in targets.iterrows():
    gns = row["Mapped gene"].split(", ")
    for gn in gns:
        autism_genes.add(gn)

autism_genes = sorted(list(autism_genes))

In [25]:
gene_mapping = mg.querymany(autism_genes, scopes=["symbol", "alias"], species="human", returnall=True)
gene_mapping["out"][:3]

querying 1-565...done.
Finished.
151 input query terms found dup hits:
	[('ACTG1P22', 2), ('ADAMTSL4-AS2', 2), ('ATP2B2', 2), ('BORCS7-ASMT', 2), ('C2', 2), ('CACNA1C-IT3',
11 input query terms found no hit:
	["'-", 'CSDE1; No mapped genes', 'CSDE1; No mapped genes; No mapped genes', 'MYO9B; No mapped genes'


[{'query': "'-", 'notfound': True},
 {'query': 'AACSP1',
  '_id': '729522',
  '_score': 41.714447,
  'entrezgene': '729522',
  'name': 'acetoacetyl-CoA synthetase pseudogene 1',
  'symbol': 'AACSP1',
  'taxid': 9606},
 {'query': 'ABCC10',
  '_id': '89845',
  '_score': 87.69568,
  'entrezgene': '89845',
  'name': 'ATP binding cassette subfamily C member 10',
  'symbol': 'ABCC10',
  'taxid': 9606}]

In [26]:
ncbi_ids = []
gene2ncbi = {}
for gn in tqdm(autism_genes):
    res = [e for e in gene_mapping["out"] if e["query"]==gn]
    res = [r for r in res if "_score" in r.keys() and "entrezgene" in r.keys()]
    if len(res)<1:
#         ncbi_ids.append(None)
        continue
#     elif len(res)>1:
#         break
        
    res = sorted(res, key=lambda x: x["_score"], reverse=True)
    ncbi_ids.append(res[0]["entrezgene"])
    gene2ncbi[gn] = res[0]["entrezgene"]
len(ncbi_ids)

  0%|          | 0/565 [00:00<?, ?it/s]

552

In [30]:
with open("processed_data/gwas_catalog_targets/autism_targets_ncbi_gwas_catalog.json", "w") as f:
    json.dump(ncbi_ids, f)
with open("processed_data/gwas_catalog_targets/autism_targets_gene2ncbi.json", "w") as f:
    json.dump(gene2ncbi, f, indent=2)
# ncbi_ids

# Schizophrenia

In [27]:
targets = pd.read_csv("data/GWAS_catalog/schizophrenia/efotraits_MONDO_0005090-associations-2022-03-25.csv")
targets

,Variant and risk allele,P-value,P-value annotation,RAF,OR,Beta,CI,Mapped gene,Reported trait,Trait(s),Background trait(s),Study accession,Location
0,rs11265461-<b>C</b>,2 x 10-7,NaN,0.41,1.45,'-,[1.26-1.67],"SLAMF1, SETP9",Schizophrenia (treatment resistant),treatment refractory schizophrenia,'-,GCST001458,1:160660353
1,rs230529-<b>T</b>,2 x 10-7,NaN,0.47,1.45,'-,[1.26-1.66],NFKB1,Schizophrenia (treatment resistant),treatment refractory schizophrenia,'-,GCST001458,4:102536261
2,rs2237457-<b>T</b>,6 x 10-7,(Recessive model),0.36,1.74,'-,[NR],GRB10,Schizophrenia (treatment resistant),treatment refractory schizophrenia,'-,GCST002604,7:50658447
3,rs2269372-<b>A</b>,4 x 10-8,NaN,NR,1.313,'-,[NR],RENBP,Schizophrenia,schizophrenia,'-,GCST002190,X:153942092
4,rs7597593-<b>T</b>,9 x 10-11,NaN,NR,1.066,'-,[1.05-1.09],ZNF804A,Schizophrenia,schizophrenia,'-,GCST004946,2:184668853
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3844,rs7193263-<b>?</b>,6 x 10-11,NaN,'-,'-,'-,'-,RBFOX1,"Anorexia nervosa, attention-deficit/hyperactiv...","anorexia nervosa, obsessive-compulsive disorde...",'-,GCST009600,16:6265879
3845,rs7200826-<b>?</b>,1 x 10-8,NaN,'-,'-,'-,'-,SHISA9,"Anorexia nervosa, attention-deficit/hyperactiv...","anorexia nervosa, obsessive-compulsive disorde...",'-,GCST009600,16:12972976
3846,rs7405404-<b>?</b>,1 x 10-12,NaN,'-,'-,'-,'-,'-,"Anorexia nervosa, attention-deficit/hyperactiv...","anorexia nervosa, obsessive-compulsive disorde...",'-,GCST009600,Mapping not available
3847,rs7521492-<b>?</b>,1 x 10-9,NaN,'-,'-,'-,'-,'-,"Anorexia nervosa, attention-deficit/hyperactiv...","anorexia nervosa, obsessive-compulsive disorde...",'-,GCST009600,1:163758686


In [28]:
sch_genes = set()
for i, row in targets.iterrows():
    gns = row["Mapped gene"].split(", ")
    for gn in gns:
        sch_genes.add(gn)

sch_genes = sorted(list(sch_genes))

In [29]:
gene_mapping = mg.querymany(sch_genes, scopes=["symbol", "alias"], species="human", returnall=True)
gene_mapping["out"][:3]

querying 1-1000...done.
querying 1001-1612...done.
Finished.
423 input query terms found dup hits:
	[('ACTG1P15', 2), ('ACTG1P22', 2), ('ADAMTSL4-AS2', 2), ('ADI1P2', 2), ('AHCYP3', 2), ('ALDH7A1P4', 
1 input query terms found no hit:
	["'-"]


[{'query': "'-", 'notfound': True},
 {'query': 'ABCA6',
  '_id': '23460',
  '_score': 95.53044,
  'entrezgene': '23460',
  'name': 'ATP binding cassette subfamily A member 6',
  'symbol': 'ABCA6',
  'taxid': 9606},
 {'query': 'ABCA9',
  '_id': '10350',
  '_score': 96.009,
  'entrezgene': '10350',
  'name': 'ATP binding cassette subfamily A member 9',
  'symbol': 'ABCA9',
  'taxid': 9606}]

In [31]:
ncbi_ids = []
gene2ncbi = {}
for gn in tqdm(autism_genes):
    res = [e for e in gene_mapping["out"] if e["query"]==gn]
    res = [r for r in res if "_score" in r.keys() and "entrezgene" in r.keys()]
    if len(res)<1:
#         ncbi_ids.append(None)
        continue
#     elif len(res)>1:
#         break
        
    res = sorted(res, key=lambda x: x["_score"], reverse=True)
    ncbi_ids.append(res[0]["entrezgene"])
    gene2ncbi[gn] = res[0]["entrezgene"]
len(ncbi_ids)

  0%|          | 0/565 [00:00<?, ?it/s]

439

In [32]:
with open("processed_data/gwas_catalog_targets/schizophrenia_targets_ncbi_gwas_catalog.json", "w") as f:
    json.dump(ncbi_ids, f)
with open("processed_data/gwas_catalog_targets/schizophrenia_targets_gene2ncbi.json", "w") as f:
    json.dump(gene2ncbi, f, indent=2)